<a href="https://colab.research.google.com/github/zerxe/CEIABDAT/blob/main/SNS/SNS_Actividad_2_1_%E2%80%93_An%C3%A1lisis_de_observaciones_influyentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad 2.1 – Análisis de observaciones influyentes
Adolfo Morales Colmenares

In [39]:
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler  # Escalamiento Z
from sklearn.preprocessing import MinMaxScaler    # Escalamiento por rangos

## Observaciones influyentes

a) (1 Punto) Calcular la media y la mediana antes de realizar la modificación de
incluir unos ingresos de 500.000€

In [31]:
datos = 15000+np.random.rand(100)*10000
datos_ord = datos
datos_ord.sort()

media=  np.mean(datos)
mediana=  np.median (datos) # Es el valor central cuando los datos se ordenan
print(f" Media= {media} y Mediana={mediana}")  # Aqui se muestra la media y mediana antes de incluir los ingresos de 500.000€
datos[50]=500000

 Media= 19960.38484808487 y Mediana=20127.330313627794


b) (2 Puntos) Aplicar el método de Probabilidad global, para detectar los outliers

Para este punto se ajusta el método de probabilidad global usando el criterio Chauvenet para tener un ajuste en la localización de outlier que sirva con los datos de esta actividad

In [32]:
# Probabilidad de la muestra de estar dentro de las bandas
p_g = 0.95
# probabilidad global
alfa_g = (1 - p_g) / 2
# probabilidad  para un solo dato
alfa= 1-(1-alfa_g)**(1/len(datos)) # Se realiza este ajuste para ser más precisos.
#alfa = alfa_g


alfa=st.norm.ppf(1-alfa/2) #Criterio Chauvenet

alfa = round(alfa, 5)

xL = round(np.mean(datos) - alfa *  np.std(datos), 4)
xU = round(np.mean(datos) + alfa *  np.std(datos), 4)

for i in  range(len(datos)):
        if datos[i] < xL  or datos[i]>xU:
          print(f" El dato[{i}]={datos[i]} es un outlier")

 El dato[50]=500000.0 es un outlier


c) (1 Punto) Repetir el mismo procedimiento de detectar los outliers para la
mediana: ¿Qué ocurre?

In [33]:
xLMean = round(np.mean(datos) - alfa *  np.mean(datos), 4)
xUMean = round(np.mean(datos) + alfa *  np.mean(datos), 4)

for i in  range(len(datos)):
        if datos[i] < xLMean  or datos[i]>xUMean:
          print(f" El dato[{i}]={datos[i]} es un outlier")


 El dato[50]=500000.0 es un outlier


En mi caso no veo cambios

## Escalamiento de datos

Considerar que la variable X toma los valores 1,2,3,4,5,6,7,8,9,10.


a) (2 Puntos) ¿Cuánto vale la media, mediana, la desviación estándar muestral, la
varianza muestral y el rango de la variable X?

In [34]:
x = np.array([1,2,3,4,5,6,7,8,9,10])

media = np.mean(x)
mediana = np.median(x)
desviacion_estandar = np.std(x, ddof=1)  # ddof=1 para cálculo muestral
varianza = np.var(x, ddof=1)  # ddof=1 para cálculo muestral
rango = np.ptp(x)  # Diferencia entre el máximo y el mínimo

print(f"Media: {media}")
print(f"Mediana: {mediana}")
print(f"Desviación estándar muestral: {desviacion_estandar}")
print(f"Varianza muestral: {varianza}")
print(f"Rango: {rango}")

Media: 5.5
Mediana: 5.5
Desviación estándar muestral: 3.0276503540974917
Varianza muestral: 9.166666666666666
Rango: 9


b) (1 Punto) Utilizar la función describe() de Panda, para obtener la media,
desviación estándar, etc…

In [35]:
print(pd.DataFrame(x).describe())

              0
count  10.00000
mean    5.50000
std     3.02765
min     1.00000
25%     3.25000
50%     5.50000
75%     7.75000
max    10.00000


c) (1 Punto)¿Por qué el resultado de calcular la desviación estándar con Numpy
es diferente a la calculada por describe de Panda? ¿Qué ajuste sería necesario
realizar para que los resultados fuesen similares/iguales?


La diferencia entre la desviación estándar calculada por Numpy y Pandas en sus configuraciones por defecto se debe al uso del divisor en la fórmula de la desviación estándar:

Numpy (np.std): Por defecto, utiliza
𝑛
 (tamaño de la muestra) como divisor, calculando la desviación estándar poblacional. Para calcular la desviación estándar muestral, se debe especificar el argumento ddof=1, lo que cambia el divisor a
𝑛
−
1

Pandas (describe): Por defecto, calcula la desviación estándar muestral, usando
𝑛
−
1 como divisor.

Ajuste necesario
Para que los resultados sean iguales entre ambos:

En Numpy, se debe usar ddof=1 al calcular la desviación estándar para obtener el resultado muestral.
Alternativamente, si deseas que Pandas calcule la desviación estándar poblacional, puedes configurar ddof=0.

In [36]:
# Cálculo con numpy (muestral)
std_numpy_muestral = np.std(x, ddof=1)

# Cálculo con pandas
std_pandas = pd.Series(x).std()

print(f"Desviación estándar con Numpy (muestral): {std_numpy_muestral}")
print(f"Desviación estándar con Pandas: {std_pandas}")


Desviación estándar con Numpy (muestral): 3.0276503540974917
Desviación estándar con Pandas: 3.0276503540974917


d) (1 Punto) Estandarizar la variable (escalamiento) mediante rangos y a
continuación calcular la media y la mediana de la variable escalada.


In [37]:
Data = pd.DataFrame(x, columns=['X'])

# Escalamiento
# ==============================================================================
# Selección de columnas numéricas
numeric_cols = Data.select_dtypes(include=['float64', 'int']).columns.to_list()

# MinMaxScaler (por rangos)
preprocessor_RR = ColumnTransformer(
    [('scale', MinMaxScaler(), numeric_cols)], remainder='passthrough'
)
Data_prep_RR = preprocessor_RR.fit_transform(Data)

# Creación de DataFrame
Data_scaled_Range = pd.DataFrame(Data_prep_RR, columns=numeric_cols)

# Cálculo de media y mediana de las variables escaladas
media_Range = Data_scaled_Range.mean()
mediana_Range = Data_scaled_Range.median()

# Resultados
print("\nMedia y mediana con escalamiento por rangos:")
print(media_Range)
print(mediana_Range)



Media y mediana con escalamiento por rangos:
X    0.5
dtype: float64
X    0.5
dtype: float64


e) (1 Punto) Repetir el apartado anterior con el escalamiento Z - score

In [38]:
# Standard scaler (Z-score)
preprocessor_SS = ColumnTransformer(
    [('scale', StandardScaler(), numeric_cols)], remainder='passthrough'
)
Data_prep_SS = preprocessor_SS.fit_transform(Data)

# Creación de DataFrame
Data_scaled_Z = pd.DataFrame(Data_prep_SS, columns=numeric_cols)

# Cálculo de media y mediana de las variables escaladas
media_Z = Data_scaled_Z.mean()
mediana_Z = Data_scaled_Z.median()

# Resultados
print("Media y mediana con escalamiento Z-score:")
print(media_Z)
print(mediana_Z)

Media y mediana con escalamiento Z-score:
X   -6.661338e-17
dtype: float64
X    0.0
dtype: float64
